In [1]:
from dotenv import load_dotenv
from typing import TypedDict, Annotated
from langgraph.graph import START, END, StateGraph
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from langgraph.graph.message import add_messages

from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import ToolNode, tools_condition

import requests
import random

In [2]:
load_dotenv()

True

In [3]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

In [46]:
# Creating Tools
search_tool = DuckDuckGoSearchRun()


# Custom Calulator Tool
@tool
def calculator(first_num: float, second_num: float, operation: str) -> dict:
    """
    Perform a basic arithmetic operation on two numbers.
    Supported operations are add, subtract, multiply, divide
    """

    try:
        if operation == "add":
            result = first_num + second_num
        elif operation == "subtract":
            result = abs(first_num - second_num)
        elif operation == "multiply":
            result = round(first_num * second_num, 2)
        elif operation == "divide":
            if second_num == 0:
                result = 0
            result = round(first_num / second_num, 2)
        else:
            return {"Error": f"Invalid Operation{operation}"}

        return {
            "first_num": first_num,
            "second_num": second_num,
            "operation": operation,
            "result": result,
        }
    except Exception as e:
        return {"error": str(e)}


# get stock price
@tool
def get_stock_price(symbol: str) -> dict:
    """
    Get the currect stock price of the provided symbol of Indian Stock Exchange
    """

    url = f"https://api.freeapi.app/api/v1/public/stocks/{symbol}"

    headers = {"accept": "application/json"}
    response = requests.get(url=url, headers=headers)

    return response.json()

In [47]:
# List of tools
all_tools = [search_tool, calculator, get_stock_price]
# Bind all tools with llm
llm_with_tool = llm.bind_tools(all_tools)

In [48]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [49]:
# graph nodes
def chat_node(state: ChatState) -> ChatState:
    """LLM node that may answer or request tool call"""

    messages = state["messages"]
    response = llm_with_tool.invoke(messages)

    return {"messages": [response]}


# *********************************************

# Tool Node : It will have all tools
tool_node = ToolNode(all_tools)

In [63]:
# create graph
graph = StateGraph(ChatState)
graph.add_node("chat_node", chat_node)

# We have to make the node name for tool node as tools because tool_condition explicitly have the literal end & tools so that it can identify which node to go
graph.add_node("tools", tool_node)

In [64]:
# edges

graph.add_edge(START, "chat_node")

"""
If LLM asked for tool, then go to tool node else finish
"""

graph.add_conditional_edges("chat_node", tools_condition)


In [65]:
# Add another node from tool to chat node so that outputs are polish and we can perform multistep tool calling
graph.add_edge("tools", "chat_node")

In [66]:
workflow = graph.compile()

In [59]:
# workflow

> Now see the problems in the output. Some outputs are not structured so that Human can understand it. And our graph end after the tool, But we need to get the output of the tool to go to chat node so that we can polish its output.

> If we need 2 tools in a single call then we cannot do that suppose search stock price and calculate price to buy x amount of share so it will not work. Because it will direct end after the tool output and cannot get to 2nd tool.


**So we have to create a node from tools to chat Node**


In [74]:
chat_output = workflow.invoke(
    {"messages": [HumanMessage(content="What is the stock price of RELIANCE")]}
)

In [68]:
print(chat_output)

{'messages': [HumanMessage(content='What is the stock price of RELIANCE', additional_kwargs={}, response_metadata={}, id='fb12f122-012e-4d8b-a801-471b7850f503'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qf86e9whs', 'function': {'arguments': '{"symbol":"RELIANCE"}', 'name': 'get_stock_price'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 440, 'total_tokens': 457, 'completion_time': 0.035342374, 'prompt_time': 0.036509896, 'queue_time': 0.057968813, 'total_time': 0.07185227}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--06dd278a-a277-4d4f-a786-afdddae9c99a-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'RELIANCE'}, 'id': 'qf86e9whs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 440, 'output_tokens': 17, 'total_tokens': 457}), ToolMessage(content='{"statusCode": 200

In [69]:
print(chat_output["messages"])

[HumanMessage(content='What is the stock price of RELIANCE', additional_kwargs={}, response_metadata={}, id='fb12f122-012e-4d8b-a801-471b7850f503'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'qf86e9whs', 'function': {'arguments': '{"symbol":"RELIANCE"}', 'name': 'get_stock_price'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 440, 'total_tokens': 457, 'completion_time': 0.035342374, 'prompt_time': 0.036509896, 'queue_time': 0.057968813, 'total_time': 0.07185227}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--06dd278a-a277-4d4f-a786-afdddae9c99a-0', tool_calls=[{'name': 'get_stock_price', 'args': {'symbol': 'RELIANCE'}, 'id': 'qf86e9whs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 440, 'output_tokens': 17, 'total_tokens': 457}), ToolMessage(content='{"statusCode": 200, "data": {"N

In [70]:
print(chat_output["messages"][-1])

content='The current stock price of RELIANCE is ₹ 2,597.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 636, 'total_tokens': 652, 'completion_time': 0.036170634, 'prompt_time': 0.052053884, 'queue_time': 0.491666194, 'total_time': 0.088224518}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_155ab82e98', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--e45024e1-b2f2-4d14-8e7c-2043043f93ed-0' usage_metadata={'input_tokens': 636, 'output_tokens': 16, 'total_tokens': 652}


In [71]:
print(chat_output["messages"][-1].content)

The current stock price of RELIANCE is ₹ 2,597.
